# Star Wars data
All data from http://swapi.co using their python API.

In [3]:
import swapi

(`swapi` can be installed with `pip install --user`.)

## Code 

`swapi.get_all(category)` pulls the data from the database and wraps it into a `Model`. The code below takes the `Model` apart and selects records of interest. Refer to the API definition for all available records. Note that models can be linked, e.g., a `homeplanet` of a person in `people` is a `Planet` class, which can in turn be interrogated.

In [48]:
class Slurper(object):
    """Handle SWAPI database and support export"""
    
    def __init__(self, category):
        self.category = category
        self.container = swapi.get_all(category)
        self.data = list(self.container.iter())
        self.records = self.get_records()
        
    def get_records(self):
        """Records are hard coded for each category"""      
        return [tuple(self._fix_entry(s) for s in rec) 
                            for rec in self._get_records_for(self.category)]
    
    def _get_records_for(self, category):
        """Internal method to select appropriate record creator"""
        get_method = {
            'planets': self.planet_records,
            'people': self.people_records,
            'vehicles': self.vehicle_records,
            'starships': self.starship_records,
        }
        return get_method[category]()
    
    def export_csv(self, fn):
        with open(fn, "w") as out:
            for rec in self.records:
                out.write(",".join(rec) + '\n')
    
    def _fix_entry(self, entry):
        """Replace comma with space and strip trailing whitespace.
        
        Also need to get rid of all special characters.
        """
        # python 2
        #return entry.replace(",", "/").strip().encode('ascii', 'ignore')
        # python 3 might handle unicode
        entry = self._fix_numbers(entry)
        return entry.replace(",", "/").strip()
    
    @staticmethod
    def _fix_numbers(x):
        """Return a number as a string if it can be interpreted as one (e.g 1,234 == 1234)"""
        s = str(x).replace(",", "").replace(" ","")
        try:
            return str(int(s))
        except ValueError:
            try:
                return str(float(s))
            except ValueError:
                pass
        return x
        
    def planet_records(self):
        """name, diameter, terrain"""
        return ((p.name, p.diameter, p.terrain) for p in self.data)
    
    def people_records(self):
        """name, homeworld, eye_color, mass"""
        return ((p.name, p.get_homeworld().name, p.eye_color, p.mass) for p in self.data)
    
    def vehicle_records(self):
        """name, model, vehicle_class, max_atomspheric_speed, cost_in_credits"""
        return ((v.name, v.model, v.vehicle_class, v.max_atmosphering_speed, 
                 v.cost_in_credits) for v in self.data)
    
    def starship_records(self):
        """name, model, vehicle_class, max_atomspheric_speed, cost_in_credits, length"""
        return ((v.name, v.model, v.starship_class, v.max_atmosphering_speed, 
          v.cost_in_credits, v.length) for v in self.data)
    

Save CSV files in the data directory:

In [22]:
%mkdir data

mkdir: data: File exists


## Extract data for all categories
Data are dumped as CSV files. For the class there is also a simple "planets.dat" file, which was generated with the script `planets_list.py`.

### Planets 

In [52]:
planets = Slurper("planets")
planets.export_csv("data/planets.csv")

### People 

In [49]:
people = Slurper("people")
people.export_csv("data/people.csv")

### Vehicles 

In [50]:
vehicles = Slurper("vehicles")
vehicles.export_csv("data/vehicles.csv")

### Starships 

In [51]:
starships = Slurper("starships")
starships.export_csv("data/starships.csv")

## Example data 

In [28]:
people.data[:5]

[<Person - Luke Skywalker>,
 <Person - C-3PO>,
 <Person - R2-D2>,
 <Person - Darth Vader>,
 <Person - Leia Organa>]

In [30]:
planets.data[:5]

[<Planet - Alderaan>,
 <Planet - Yavin IV>,
 <Planet - Hoth>,
 <Planet - Dagobah>,
 <Planet - Bespin>]

In [32]:
starships.data[:5]

[<Starship - Sentinel-class landing craft>,
 <Starship - Death Star>,
 <Starship - Millennium Falcon>,
 <Starship - Y-wing>,
 <Starship - X-wing>]